Project 1: Sudoku

Group 5

Names: Connor Munson, Nick Kasten, Dayton Wickerd

Date: September 12th 2025


In [248]:
#Parameters
GROUP_ID = 5
ALGORITHM = "sa"
PUZZLE_TYPE = "easy"
PUZZLE_PATH = "Puzzles/Easy-P1.txt"


In [249]:
#Imports
import logging as log
import random
import math
import copy
import itertools
import re

This is the cell class which represents individual spaces in the puzzle board.

In [250]:
class Cell:

  def __init__(self, val: int, domain: list, given: bool):
    self.val = val
    self.domain = domain
    self.given = given

  # Adding a value to the domain
  def domainAdd(self, val):
    self.domain.append(val)

  # Removing a value from the domain
  def domainStrip(self, val):
    self.domain.remove(val)

  # Getters and Setters
  def getDomain(self):
    return self.domain

  def getVal(self):
    return self.val

  def setVal(self, val):
    self.val = val

  def isGiven(self):
    return self.given

  def neighbors(i, j):
    x = set()
    #going through the same row
    for c in range(9):
      if c != j:
        x.add ((i,c))
    #going through the same column
    for r in range(9):
      if r != i:
        x.add ((r,j))
    # same 3x3 box. y = box row, z = box column
    y, z = (i // 3) * 3, (j // 3) * 3
    for r in range(y, y + 3):
        for c in range(z, z + 3):
            if (r, c) != (i, j):
                x.add((r, c))
    return x



This is the puzzle class which has a few different methods:

createBoard - builds the array of type Cell for algorithmic use

printBoard - prints the current board state to stdout

isValid - returns true if there are no conflicts (excluding empty cells), and false otherwise

isSolved - returns true if there are no more empty cells and if the puzzle is valid, false otherwise

In [251]:
class Puzzle:

    def __init__(self, data: list):
        self.data = data
        self.board = []

    def createBoard(self):
        self.board = []
        default_domain = [1,2,3,4,5,6,7,8,9]
        for i in range(9):
            row = []
            for j in range(9):
                if self.data[i][j] != "?":
                    value = int(self.data[i][j])
                    given_domain = [value]
                    row.append(Cell(value, given_domain, True))
                else:
                    # blank cells (?) are repalced with a 0
                    row.append(Cell(0, default_domain, False))
            self.board.append(row)

    def printBoard(self):
        print("Current Board State:")
        print("- - - - - - - - -")
        for i in range(9):
            row_vals = []
            for j in range(9):
                row_vals.append(str(self.board[i][j].getVal()))
            print(" ".join(row_vals))
        print("- - - - - - - - -")

    def printSolvedBoard(self):
        print("Solved Board State:")
        print("- - - - - - - - -")
        for i in range(9):
            row_vals = []
            for j in range(9):
                row_vals.append(str(self.board[i][j].getVal()))
            print(" ".join(row_vals))
        print("- - - - - - - - -")

    # Checking if a single row is valid
    def isRowValid(self, row) -> bool:
      seen = set()
      for j in range(9):
          val = self.board[row][j].getVal()
          if val == 0:
              continue
          if val in seen:
              #print(f"Invalid row {row}: duplicate value {val}")
              return False
          seen.add(val)
      return True

    # Checking if a single column is valid
    def isColValid(self, col) -> bool:
        seen = set()
        for i in range(9):
            val = self.board[i][col].getVal()
            if val == 0:
                continue
            if val in seen:
                #print(f"Invalid column {col}: duplicate value {val}")
                return False
            seen.add(val)
        return True

    # Checking if a single box is valid
    def isBoxValid(self, startRow, startCol) -> bool:
        seen = set()
        for i in range(3):
            for j in range(3):
                val = self.board[startRow + i][startCol + j].getVal()
                if val == 0:
                    continue
                if val in seen:
                    #print(f"Invalid box at ({startRow}, {startCol}): duplicate value {val}")
                    return False
                seen.add(val)
        return True


    # Finds the next empty cell (cell = 0)
    def findNextEmpty(self):
      for i in range(9):
        for j in range(9):
          if self.board[i][j].getVal() == 0:
            return i, j
      return None, None

    # All encompasing validity check
    def isValid(self) -> bool:
      # Checks all rows
      for i in range(9):
          if not self.isRowValid(i):
              return False

      # Checks all columns
      for j in range(9):
          if not self.isColValid(j):
              return False

      # Checks all 3x3 boxes
      for row in range(0, 9, 3):
          for col in range(0, 9, 3):
              if not self.isBoxValid(row, col):
                  return False

      return True

    def isSolved(self) -> bool:
      i, j = self.findNextEmpty()
      print(f"Is Valid: {self.isValid()}")
      print(f"Next Empty: {i},{j}")
      if self.isValid() and i == None and j == None:
        return True
      else:
        return False

    def getGivens(self):
      for i in range(9):
        for j in range(9):
          if self.board[i][j].checkGiven() == True:
            print("Given: ", self.board[i][j].getVal(), "Domain: ", self.board[i][j].getDomain())

    def out(self, path):
       with open(path, "w") as file:
          for i in range(9):
             for j in range(9):
                file.write(str(self.board[i][j].getVal()) + ",")
             file.write("\n")


This function takes the path of the input puzzle and converts it to type Puzzle


In [252]:
def puzzleImporter(path) -> Puzzle:
  try:
    board = []
    with open(path, "r", encoding="utf-8-sig") as f:
      for line in f:
            row = [x.strip() for x in line.split(",")]
            board.append(row)
    try:
      new_puzzle = Puzzle(board)
      return new_puzzle
    except Exception as e:
      print(f"Error: {e}")
  except FileNotFoundError:
      print("Error: The file " + path +  " was not found.")

This is the simple, brute-force backtracking algorithm using depth first search.


In [253]:
def backTracking(p: Puzzle, counter) -> Puzzle:

  if 'decisions' not in counter:
    counter['decisions'] = 0
    counter['checks'] = 0
    counter['backtracks'] = counter.get('count', 0) #keeps original checks we had before making this


  possible_values = [1,2,3,4,5,6,7,8,9]

  row, col = p.findNextEmpty()
  if row == None and col == None:
    print(f"[bt] decisions={counter['decisions']} checks={counter['checks']} backtracks={counter['backtracks']}")
    return p

  for num in possible_values:
    p.board[row][col].setVal(num)
    counter['decisions'] += 1 # attempt an assignment, count as a decision

    counter['checks'] += 1 # check constraints once per attempt
    if p.isValid():
      if backTracking(p, counter) != None:
        return p

    # if we get here, then we hit a contradiction and count backtrack here
    p.board[row][col].setVal(0) #back track here
    counter['backtracks'] += 1
    counter['count'] += 1

  return None

Backtracking with forward checking.

In [254]:
def forwardChecking(p: Puzzle, counter) -> Puzzle:
    
    # metrics
    if 'fc_initialized' not in counter:
       counter['fc_initialization'] = True
       counter['decisions'] = counter.get('decisions', 0)
       counter['checks'] = counter.get('checks', 0)
       counter['backtracks'] = counter.get('backtracks', counter.get('count', 0))
       counter['prunes'] = counter.get('prunes', 0)

    # helper functions
    def neighbors(i, j):
        # x = set of neighbors, r = row, c = column
        x = set()
        # same row
        for c in range(9):
            if c != j:
                x.add((i, c))
        # same column
        for r in range(9):
            if r != i:
                x.add((r, j))
        # same 3x3 box. y = box row, z = box column
        y, z = (i // 3) * 3, (j // 3) * 3
        for r in range(y, y + 3):
            for c in range(z, z + 3):
                if (r, c) != (i, j):
                    x.add((r, c))
        return x

    # propagates the domains of neighbor cells
    def initialPropagation():
      for r in range(9):
        for c in range(9):
          cell = p.board[r][c]

          if cell.getVal() != 0:
            value = cell.getVal()

            for nr, nc in neighbors(r, c):
              neighbor_cell = p.board[nr][nc]
              if value in neighbor_cell.getDomain():
                try:
                  neighbor_cell.domainStrip(value)
                  counter['prunes'] += 1 # count domain removals
                except ValueError:
                  pass

    def initializeDomains():
        for r in range(9):
            for c in range(9):
                cell = p.board[r][c]
                if cell.getVal() != 0:
                    # Given cells have single-value domains
                    cell.domain = [cell.getVal()]
                else:
                    # Empty cells start with full domain
                    cell.domain = [1, 2, 3, 4, 5, 6, 7, 8, 9]

    def propagateAssignment(row, col, value):
      removed_values = []

      for nr, nc in neighbors(row, col):
        neighbor_cell = p.board[nr][nc]
        if neighbor_cell.getVal() == 0 and value in neighbor_cell.getDomain():
          try:
            neighbor_cell.domainStrip(value)
            removed_values.append((nr, nc, value))
            counter['prunes'] += 1 # count each prune


            if len(neighbor_cell.getDomain()) == 0:
              return False, removed_values

          except ValueError:
            pass

      return True, removed_values

    def undoPropagation(removed_values):
      for row, col, value in removed_values:
        try:
          p.board[row][col].domainAdd(value)
        except ValueError:
          pass

    def backTracking_FC(counter):

        row, col = p.findNextEmpty()
        if row == None and col == None:
          print(f"[fc] decisions={counter.get('decisions', 0)} "
                f"checks={counter.get('checks', 0)} "
                f"backtracks={counter.get('backtracks', counter.get('count', 0))} "
                f"prunes={counter.get('prunes', 0)}")
          return p

        domain_values = list(p.board[row][col].getDomain())


        for num in domain_values:
          p.board[row][col].setVal(num)
          counter['decisions'] = counter.get('decisions', 0) + 1

          success, removed_values = propagateAssignment(row, col, num)

          # if success and p.isValid():
          #   if backTracking_FC(counter) != None:
          #     return p

          if success:
             counter['checks'] = counter.get('checks', 0) + 1
             if p.isValid():
                if backTracking_FC(counter) is not None:
                   return p

          # if we get here, then we hit a contradiction and count backtrack here
          p.board[row][col].setVal(0) #back track here
          undoPropagation(removed_values)
          counter['backtracks'] = counter.get('backtracks', counter.get('count', 0)) + 1
          counter['count'] = counter.get('count', 0) + 1      

        return None

    initializeDomains()
    initialPropagation()

    for r in range(9):
      for c in range(9):
        if p.board[r][c].getVal() == 0 and len(p.board[r][c].getDomain()) == 0:
          return None

    if backTracking_FC(counter) != None:
      return p
    else:
      return None



Backtracking with Arc Consistency (AC3).

In [255]:
def arcConsistency(p: Puzzle, counter) -> Puzzle:
    
    if 'ac3_initialized' not in counter:
        counter['ac3_initialized'] = True
        counter['decisions']  = counter.get('decisions', 0) # attempted assignments
        counter['checks']     = counter.get('checks', 0) # calls isValid()
        counter['backtracks'] = counter.get('backtracks', counter.get('count', 0))
        counter['prunes']     = counter.get('prunes', 0) # domain values removed
        counter['revisions']  = counter.get('revisions', 0) 

    from queue import Queue

    # helper functions
    def neighbors(i, j):
        # x = set of neighbors, r = row, c = column
        x = set()
        # same row
        for c in range(9):
            if c != j:
                x.add((i, c))
        # same column
        for r in range(9):
            if r != i:
                x.add((r, j))
        # same 3x3 box. y = box row, z = box column
        y, z = (i // 3) * 3, (j // 3) * 3
        for r in range(y, y + 3):
            for c in range(z, z + 3):
                if (r, c) != (i, j):
                    x.add((r, c))
        return x

    def REVISE(Xi, Xj):
        ri, ci = Xi
        rj, cj = Xj

        Di = list(p.board[ri][ci].getDomain())
        Dj = p.board[rj][cj].getDomain()

        revised = False
        removed_in_this_call = 0
        # from textbook: "if no value y in Dj allows (x,y) to satisfy the
        # constraint between Xi and Xj then
        # delete x from Di"
        for x in Di:
          hasConstVal = False
          for y in Dj:
            if x != y:
              hasConstVal = True # values MUST be different
              break
          if not hasConstVal:
            try:
              p.board[ri][ci].domainStrip(x)
              revised = True
              removed_in_this_call += 1
              counter['prunes'] += 1
            except ValueError:
              pass # if value is already removed

        if removed_in_this_call > 0:
           counter['revisions'] += 1

        return revised

    # initiate domains
    for r in range(9):
      for c in range(9):
        cell = p.board[r][c]
        # create domian copy
        cell.domain = list(cell.getDomain())
        val = cell.getVal()
        if val != 0:
          cell.domain = [val]


    # AC3, following the logic from the textbook (p. 171)
    q = Queue()
    for i in range(9):
        for j in range(9):
            for n in neighbors(i, j):
                q.put(((i, j), n))

    while not q.empty():
        (Xi, Xj) = q.get()
        if REVISE(Xi, Xj):
            ri, ci = Xi
            if len(p.board[ri][ci].getDomain()) == 0:
                break # try returning to backtracking instead
                #return p # not sure if this is correct?
            for Xk in neighbors(ri, ci):
                if Xk != Xj:
                    q.put((Xk, Xi))

    # if the domain is just 1 integer, then we should know the value and put it
    # into the puzzle.
    # ---------------
    for i in range(9):
        for j in range(9):
            cell = p.board[i][j]
            d = cell.getDomain()
            if cell.getVal() == 0 and len(d) == 1 and (not cell.isGiven()):
                k = d[0]
                cell.setVal(k)
                cell.domain = [k]
    # ---------------

    # BACKTRACKING
    def backTracking_AC3(counter):
      row, col = p.findNextEmpty()
      if row == None and col == None:
        print(f"[ac3] decisions={counter.get('decisions', 0)} "
        f"checks={counter.get('checks', 0)} "
        f"backtracks={counter.get('backtracks', counter.get('count', 0))} "
        f"prunes={counter.get('prunes', 0)} "
        f"revisions={counter.get('revisions', 0)}")
        return p # solved

      # use domain from ac3
      ac3_domain = p.board[row][col].getDomain()

      for n in ac3_domain:
        p.board[row][col].setVal(n)

        counter['decisions'] = counter.get('decisions', 0) + 1
        counter['checks'] = counter.get('checks', 0) + 1

        if p.isValid():
          recurse = backTracking_AC3(counter)
          if recurse is not None:
            return recurse

        p.board[row][col].setVal(0)
        counter['backtracks'] = counter.get('backtracks', counter.get('count', 0)) + 1
        counter['count'] = counter.get('count', 0) + 1 

      return None

    result = backTracking_AC3(counter)
    return result if result is not None else p


In [256]:
def simulatedAnnealing(p: Puzzle, max_iter=1000, temp=1.0, cooling=0.995) -> Puzzle:

  # metrics
  decisions = 0
  moves_accepted = 0
  worse_moves_accepted = 0

  def neighbors(i, j):
    x = set()
    #going through the same row
    for c in range(9):
      if c != j:
        x.add ((i,c))
    #going through the same column
    for r in range(9):
      if r != i:
        x.add ((r,j))
    # same 3x3 box. y = box row, z = box column
    y, z = (i // 3) * 3, (j // 3) * 3
    for r in range(y, y + 3):
        for c in range(z, z + 3):
            if (r, c) != (i, j):
                x.add((r, c))
    return x

  def randomFill(p):
      #this will fill all the 3x3 boxes on the board ignoring given cells
      for box_row in range(0,9,3):
        for box_col in range(0,9,3):
          nums=[1,2,3,4,5,6,7,8,9]
          for i in range(3):
            for j in range(3):
              val = p.board[box_row+i][box_col+j].getVal()
              if val != 0:
                nums.remove(val)

          for i in range(3):
            for j in range(3):
              if p.board[box_row+i][box_col+j].getVal() == 0:
                val=random.choice(nums)
                nums.remove(val)
                p.board[box_row+i][box_col+j].setVal(val)



  def minConflictSwap(p):
      #Pick a conflicted cell and swap it with another in its box to reduce conflicts.
      conflicted = []
      for i in range(9):
          for j in range(9):
              if not p.board[i][j].isGiven() and cellConflicts(p, i, j) > 0:
                  conflicted.append((i, j))

      if not conflicted:
          return None  # no conflicts left

      r, c = random.choice(conflicted)
      box_row, box_col = (r // 3) * 3, (c // 3) * 3
      best_swap = None
      best_cost = float("inf")

      for i in range(3):
          for j in range(3):
              rr, cc = box_row+i, box_col+j
              if not p.board[rr][cc].isGiven() and (rr, cc) != (r, c):
                  v1, v2 = p.board[r][c].getVal(), p.board[rr][cc].getVal()
                  # try swap
                  p.board[r][c].setVal(v2)
                  p.board[rr][cc].setVal(v1)
                  new_cost = cost(p)
                  # undo swap
                  p.board[r][c].setVal(v1)
                  p.board[rr][cc].setVal(v2)

                  if new_cost < best_cost:
                      best_cost = new_cost
                      best_swap = (rr, cc)

      if best_swap:
          rr, cc = best_swap
          v1, v2 = p.board[r][c].getVal(), p.board[rr][cc].getVal()
          p.board[r][c].setVal(v2)
          p.board[rr][cc].setVal(v1)
          return (r, c), (rr, cc)

      return None




  def randomSwap(p):
      #swaps 2 non given cells in a random 3x3 box.
      box_row = random.choice([0,3,6])
      box_col = random.choice([0,3,6])
      cells=[]

      for i in range(3):
        for j in range(3):
          if not p.board[box_row+i][box_col+j].isGiven():
            cells.append((box_row+i, box_col+j))

      if len(cells)>=2:
        (r1, c1), (r2, c2) = random.sample(cells, 2)
        v1, v2 = p.board[r1][c1].getVal(), p.board[r2][c2].getVal()
        p.board[r1][c1].setVal(v2)
        p.board[r2][c2].setVal(v1)
        return(r1, c1), (r2, c2)
      return None

  def cellConflicts(p, i, j):
      #returns the conflicts for a single cell
      val=p.board[i][j].getVal()
      if val == 0:
        return 0
      seen = 0
      for(r,c) in neighbors(i, j):
        if p.board[r][c].getVal() == val:
          seen += 1
      return seen

  def cost(p):
      total = 0
      # Row Cost
      for r in range(9):
        count = [0]*10
        for c in range(9):
          v = p.board[r][c].getVal()
          if v:
            count[v] += 1
        total += sum(c - 1 for c in count if c > 1)

      # Column Cost
      for c in range(9):
        count = [0]*10
        for r in range(9):
          v = p.board[r][c].getVal()
          if v:
            count[v] += 1
        total += sum(c - 1 for c in count if c > 1)


      # Box Cost
      for br in range(0, 9, 3):
        for bc in range(0, 9, 3):
          count = [0]*10
          for dr in range(3):
            for dc in range(3):
              v = p.board[br+dr][bc+dc].getVal()
              if v:
                count[v] += 1
          total += sum(c - 1 for c in count if c > 1)

      return total


  #use cellConflicts, cost, randomSwap, and randomFill
  randomFill(p)
  current_cost = cost(p)
  initial_cost = current_cost
  T = temp

  iters = 0
  for i in range(max_iter):
    iters = i + 1
    swap = minConflictSwap(p)
    if not swap:
      continue

    decisions += 1 # one attemped move

    new_cost = cost(p)
    delta = new_cost - current_cost

    # if delta <= 0 or random.random() < math.exp(-delta/T):
    #   current_cost = new_cost
    # else:
    #   (r1, c1), (r2, c2) = swap
    #   v1, v2 = p.board[r1][c1].getVal(), p.board[r2][c2].getVal()
    #   p.board[r1][c1].setVal(v2)
    #   p.board[r2][c2].setVal(v1)

    # T *= cooling
    # if current_cost == 0:
    #   break

    if delta <= 0:
        moves_accepted += 1
        current_cost = new_cost
    else:
        if random.random() < math.exp(-delta/T):
            moves_accepted += 1
            worse_moves_accepted += 1
            current_cost = new_cost
        else:
            # revert swap
            (r1, c1), (r2, c2) = swap
            v1, v2 = p.board[r1][c1].getVal(), p.board[r2][c2].getVal()
            p.board[r1][c1].setVal(v2)
            p.board[r2][c2].setVal(v1)
    T *= cooling
    if current_cost == 0:
       break

  acc_rate = (moves_accepted / decisions) if decisions else 0.0
  print(
      f"[sa] decisions={decisions} "
      f"moves_accepted={moves_accepted} "
      f"worse_moves_accepted={worse_moves_accepted} "
      f"accept_rate={acc_rate:.3f} "
      f"iterations={iters} "
      f"initial_cost={initial_cost} final_cost={current_cost}"
  )
  return p

In [257]:
def geneticAlgorithm(p: Puzzle, population_size, mutation_rate, selection_k, max_generations, counter=None) -> Puzzle:

  # metrics
  if counter is None:
    counter = {}
    if 'ga_initialized' not in counter:
      counter['ga_initialized'] = True
      counter['decisions'] = counter.get('decisions', 0) # number of offspring created and evaluated
      counter['generations'] = counter.get('generations', 0) # loop iterations
      counter['offspring'] = counter.get('offspring', 0) # same as decisions
      counter['fitness_evals'] = counter.get('fitness_evals', 0) # population fitness (initial pop + each child)
      counter['mutations_applied'] = counter.get('mutations_applied', 0) # each time a mutation happens

  # From first hyper param run:
  # Best hyperparameters found: {'pop_size': 50, 'mutation_rate': 0.1, 'tournament_k': 5, 'best_fitness': 26, 'generations': 50000}
  # ------------------------------------------------------------
  # Filling in the population by ensuring box validity
  # BEST FIT: pop_size=50
  def popFill(p, pop_size):
    population = []

    # Creating a new candidate until the population size is reached
    for _ in range(pop_size):
      # making a copy of the initial puzzle
      candidate = copy.deepcopy(p)

      # Traversing boxes in copy of original puzzle
      for box_row in range(0, 9, 3):
        for box_col in range(0, 9, 3):

          # Set of values present in each box
          givens = set()
          # Set of empty cells in each box
          empty_cells = []

          for i in range(3):
            for j in range(3):
              cell = candidate.board[box_row + i][box_col + j]
              # if the cell is given, add it to the set of givens
              if cell.isGiven():
                givens.add(cell.getVal())
              # if the cell is not given, add it to the set of empty values
              else:
                empty_cells.append(cell)

          # Filling in each box with non-conflicting values
          missing = [d for d in range(1, 10) if d not in givens]

          # Shuffling the values to introduce randomness
          random.shuffle(missing)

          for cell in empty_cells:
            cell.setVal(missing.pop())

      # Adding new candidate to the population
      population.append(candidate)
    return population

  # ----------------------------------------------------------------------
  # Function for evaluating the fitness of each candidate in the population
  # Decided to go with a minimization system. Since the boxes are
  # set to be valid from the outset, we are just checking the number
  # of conflicts in the rows and columns
  # F(P_i) = [num_row_conflicts + num_col_conflicts] for all candidates
  # A solved board will have a score of 0, and this will return a set of
  # scores for every candidate in the population (potentially high runtime)
  def evalFitness(population):

    # Initializing the list of candidate scores
    scores = []

    for candidate in population:
      conflicts = 0

      # Row conflicts
      for i in range(9):
        # Collecting all row values for row i
        row_vals = [candidate.board[i][j].getVal() for j in range(9) if candidate.board[i][j].getVal() != 0]
        # The length of the row - set of numbers in
        # that row (unique values) = num row conflicts
        conflicts += len(row_vals) - len(set(row_vals))

      # Column conflicts
      for j in range(9):
        # Collecting all column values for column j
        col_vals = [candidate.board[i][j].getVal() for i in range(9) if candidate.board[i][j].getVal() != 0]
        # The length of the col - set of number in
        # that col (unique values) = num col conflicts
        conflicts += len(col_vals) - len(set(col_vals))

      # Adding the candidate conflicts as an entry in the scores list
      scores.append(conflicts)
    return scores

  #--------------------------------------------------------------
  # Stop condidtions for the genetic algorithm
  # Either it finds a solution or reaches the maximum generations
  def terminatePop(population, scores, gen, max_gens=10000):

    # A zero anywhere in the population represents a solved board
    if 0 in scores:
      return True

    # Generation exceeded check
    if gen >= max_gens:
      return True

    return False

  # ------------------------------------------------------------
  # Function for selecting the parents of the population via
  # tournament selection. Using the fitness scores and the population,
  # we pick k random candidates from the population and picks the "best"
  # two candidates to be parents according to their fitness
  def selectParents(pop, scores, k=3):
    # Initializing the list of parents
    parents = []

    # Picking the parents
    for _ in range(2):
      # Getting a random sample of size k from our population
      contenders_index = random.sample(range(len(pop)), k)

      # Comparing the fitness of the candidates
      best_index = min(contenders_index, key=lambda index: scores[index])

      # Adding the best candidate from the sample
      parents.append(pop[best_index])
    return parents

  # ------------------------------------------------------------
  # Function for recombination (crossover) of the parents. We have a
  # few different crossover methods picked randomly. A row crossover,
  # a column crossover, and a box crossover. Returns a single child
  # board.
  xover_counts = {'row': 0, 'col': 0, 'box': 0}
  def recombine(parents):

    # Initializing parent puzzles
    parent1 = parents[0]
    parent2 = parents[1]

    # Initializing the child as a copy of parent 1
    child = copy.deepcopy(parent1)

    # Randomly selecting the type of crossover to
    # provide more genetic diversity in the population
    crossover_type = random.choice(['row', 'col', 'box'])
    xover_counts[crossover_type] += 1
    # Row crossover
    if crossover_type == 'row':
      # Randomly selecing a row
      row_index = random.randint(1, 8)
      for i in range(row_index, 9):
        for j in range(9):
          # Checking if the current cell is given
          if not child.board[i][j].isGiven():
            # Crossover
            child.board[i][j].setVal(parent2.board[i][j].getVal())

    # Column crossover
    elif crossover_type == 'col':
      # Randomly selecting a column
      col_index = random.randint(1, 8)
      for i in range(9):
        for j in range(col_index, 9):
          # Checking if the current cell is given
          if not child.board[i][j].isGiven():
            # Crossover of non-givens
            child.board[i][j].setVal(parent2.board[i][j].getVal())

    # Box crossover
    elif crossover_type == 'box':
      # Selecting a random box
      box_row = random.randint(0, 2) * 3
      box_col = random.randint(0, 2) * 3
      for i in range(3):
        for j in range(3):
          # Checking if the current cell is given
          if not child.board[box_row + i][box_col + j].isGiven():
            # Crossover of non-givens
            child.board[box_row + i][box_col + j].setVal(parent2.board[box_row + i][box_col + j].getVal())

    return child



  # ------------------------------------------------------------
  # Function for mutating a candidate in the population. Mutation
  # proceeds by either doing a random cell flip, a swap of two random
  # cells, or by flipping all the values in a box. None of these
  # operations will act on given cells
  def mutate(candidate, mutation_rate):
    # Making a copy of the given candidate
    child = copy.deepcopy(candidate)
    mutated = False

    # Mutating at random
    if random.random() < mutation_rate:

      mutated = True
      # Randomly selecting mutation procedure
      mutation_type = random.choice(['cell_flip', 'cell_swap', 'box_shuffle'])

      # Getting the list of all mutatable cells in the board
      all_non_givens = [cell for row in child.board for cell in row if not cell.isGiven()]

      if mutation_type == 'cell_flip':
        # Pick a random non-given cell
        cell = random.choice(all_non_givens)
        # Picking a new random value that is not the current value
        new_val = random.choice([v for v in range(1, 10) if v != cell.getVal()]) # Corrected GetVal to getVal
        cell.setVal(new_val)

      elif mutation_type == 'cell_swap':
        # Pick any two random cells
        cell1, cell2 = random.sample(all_non_givens, 2)
        val1 = cell1.getVal()
        val2 = cell2.getVal()
        # Swapping values
        cell1.setVal(val2)
        cell2.setVal(val1)

      elif mutation_type == 'box_shuffle':
        # Picking a random 3x3 box
        box_row = random.randint(0, 2) * 3
        box_col = random.randint(0, 2) * 3

        # Get the non-givens in the box
        non_givens = []
        for i in range(3):
          for j in range(3):
            cell = child.board[box_row + i][box_col + j]
            if not cell.isGiven():
              non_givens.append(cell)

        # Getting the values of the non-givens
        vals = [cell.getVal() for cell in non_givens]
        # Shuffling the list of values
        random.shuffle(vals)
        # Setting the cells to their new values
        for cell, val in zip(non_givens, vals):
          cell.setVal(val)

    return child, mutated

  # ------------------------------------------------------------
  # Simple replacement function. If the new candidate is better than
  # the worst candidate in the population, we are replacing that candidate
  # with the new candidate.
  def replace(pop, scores, child, child_fitness):
    # Find the worst candidate in the population
    worst_index = scores.index(max(scores))

    # Replace if the child is better or equal
    if child_fitness <= scores[worst_index]:
      # Replacing the worse candidate with the child
      pop[worst_index] = child
      # Replacing the score for continuity
      scores[worst_index] = child_fitness

    return pop, scores

# Flow taken from pseudocode:
  generation = 0
  population = popFill(p, pop_size=population_size)
  pop_fitness = evalFitness(population)
  counter['fitness_evals'] += len(population) 
  while not terminatePop(population, pop_fitness, generation, max_gens=max_generations):
    generation += 1
    counter['generations'] += 1
    parents = selectParents(population, pop_fitness, k=selection_k)
    child = recombine(parents)
    child, mutated = mutate(child, mutation_rate=mutation_rate)
    if mutated:
      counter['mutations_applied'] += 1
    child_fitness = evalFitness([child])[0]
    counter['fitness_evals'] += 1
    counter['offspring'] += 1
    counter['decisions'] += 1
    population, pop_fitness = replace(population, pop_fitness, child, child_fitness)
    if generation % 100 == 0:
      print(f"Best fitness of generation {generation}: {min(pop_fitness)}")

  # Find the best candidate in the final population
  best_index = pop_fitness.index(min(pop_fitness))
  best_fitness = pop_fitness[best_index]
  counter['best_fitness'] = best_fitness

  # for crossover stats:
  total_x = sum(xover_counts.values())
  pct = {k: (round(100.0 * v / total_x, 1) if total_x else 0.0) for k, v in xover_counts.items()}
  counter['xover_counts'] = xover_counts
  counter['xover_pct'] = pct
  print(f"[ga xover] counts={xover_counts} pct={pct}", flush=True) 
  
  return population[best_index]


In [258]:
def grid_search(puzzle, pop_sizes, mutation_rates, tournament_ks, max_generations):
    results = []

    for pop_size, mutation_rate, k in itertools.product(pop_sizes, mutation_rates, tournament_ks):
        best_fitness, generation = geneticAlgorithm(puzzle, pop_size, mutation_rate, k, max_generations)
        results.append({
            'pop_size': pop_size,
            'mutation_rate': mutation_rate,
            'tournament_k': k,
            'best_fitness': best_fitness,
            'generations': generation
        })

        print(f"pop_size={pop_size}, mutation_rate={mutation_rate}, k={k} => best_fitness={best_fitness}, generations={generation}")

    return results

The solver function manages the flow of algorithm selection

In [259]:
def solver(p: Puzzle, alg, diff) -> Puzzle:
  if alg == "bt" and (diff == "easy" or diff == "medium"):
    counter = {'count': 0}
    puzzle = backTracking(p, counter)
    print(f"Number of backtracks: {counter['count']}")
    return puzzle
  elif alg == "ga":
    """pop_sizes = [50]
    mutation_rates = [0.15]
    tournament_ks = [2, 3, 5, 7, 9, 11]
    max_generations = 50000
    # Tuning using Grid Search
    results = grid_search(p, pop_sizes, mutation_rates, tournament_ks, max_generations)

    # Best Result
    best_result = min(results, key=lambda r: r['best_fitness'])
    print("Best hyperparameters found:", best_result)"""
    pop_size = 500
    mutation_rate = 0.15
    tournament_k = 3
    counter = {
      'decisions': 0,
      'generations': 0,
      'offspring': 0,
      'fitness_evals': 0,
      'mutations_applied': 0, 
    }

    puzzle = geneticAlgorithm(p, population_size=pop_size, mutation_rate=mutation_rate, selection_k=tournament_k, max_generations=100000, counter=counter)
    print(
        f"[ga] decisions={counter['decisions']} "
        f"generations={counter['generations']} "
        f"offspring={counter['offspring']} "
        f"fitness_evals={counter['fitness_evals']} "
        f"mutations_applied={counter['mutations_applied']}"
    )
    return puzzle
    
  elif alg == "fc":
    counter = {'count': 0}
    puzzle = forwardChecking(p, counter)
    print(f"Number of backtracks (FC): {counter['count']}")
    return puzzle
  elif alg == "ac3":
    counter = {'count': 0}
    puzzle = arcConsistency(p, counter)
    print(f"Number of backtracks (AC3): {counter['count']}")
    return puzzle
  elif alg == "sa":
    puzzle = simulatedAnnealing(p)
    return puzzle
  else:
    log.error("Invalid Algorithm or Difficulty")


This is the wrapper


In [260]:
def wrapper():

  pattern = r"P(\d+)\.txt$"
  match = re.search(pattern, PUZZLE_PATH)
  PUZZLE_NUMBER = match.group(1)
  output_path = f"Group0{GROUP_ID}_{ALGORITHM}_{PUZZLE_TYPE}_puzzle0{PUZZLE_NUMBER}.txt"

  try:
    new_puzzle = puzzleImporter(PUZZLE_PATH)
    new_puzzle.createBoard()
    new_puzzle.printBoard()
  except Exception as e:
    log.warning("Could not create puzzle")
    print(f"An error has occured: {e}")


  solved = solver(new_puzzle, ALGORITHM, PUZZLE_TYPE)
  if solved.isSolved():
    solved.printSolvedBoard()
  else:
    solved.printBoard()

  solved.out(output_path)

wrapper()

Current Board State:
- - - - - - - - -
0 0 8 0 5 6 0 0 0
7 0 4 0 0 0 6 1 9
0 0 0 0 0 0 8 5 0
6 0 7 0 2 9 5 0 0
0 0 9 0 6 0 1 0 0
0 0 2 3 1 0 9 0 4
0 3 5 0 0 0 0 0 0
4 2 1 0 0 0 3 0 6
0 0 0 8 3 0 4 0 0
- - - - - - - - -
[sa] decisions=152 moves_accepted=139 worse_moves_accepted=2 accept_rate=0.914 iterations=152 initial_cost=40 final_cost=0
Is Valid: True
Next Empty: None,None
Solved Board State:
- - - - - - - - -
1 9 8 7 5 6 2 4 3
7 5 4 2 8 3 6 1 9
2 6 3 1 9 4 8 5 7
6 1 7 4 2 9 5 3 8
3 4 9 5 6 8 1 7 2
5 8 2 3 1 7 9 6 4
8 3 5 6 4 2 7 9 1
4 2 1 9 7 5 3 8 6
9 7 6 8 3 1 4 2 5
- - - - - - - - -
